In [1]:

import pandas as pd
from scipy.stats import kruskal

In [2]:
df = pd.read_csv("cycling.csv", sep=r"\s+")  
df["scored"] = (df["points"] > 0).astype(int)

In [3]:
df.columns

Index(['all_riders', 'rider_class', 'stage', 'points', 'stage_class',
       'scored'],
      dtype='object')

## Hypothesis A: Overall Performance Comparison
- $H_0$: All rider classes have identical distributions of scoring rates
- $H_1$: At least one rider class has a different distribution of scoring rates

In [5]:
rider_summary = (
    df.groupby("all_riders")
    .agg(
        scoring_rate=("scored", "mean"),  # % stages scored (0-1)
        rider_class=("rider_class", "first"),
    )
    .reset_index()
)
rider_summary.head(5)

,all_riders,scoring_rate,rider_class
0,Adam Yates,0.947368,All Rounder
1,Alberto Dainese,0.421053,Sprinter
2,Aleksandr Vlasov,0.473684,All Rounder
3,Alex Aranburu,0.157895,Sprinter
4,Alex Baudin,0.315789,Unclassed


In [ ]:
# Prepare data for Kruskal-Wallis
all_rounder = rider_summary[rider_summary["rider_class"] == "All Rounder"][
    "scoring_rate"
]
climber = rider_summary[rider_summary["rider_class"] == "Climber"]["scoring_rate"]
sprinter = rider_summary[rider_summary["rider_class"] == "Sprinter"]["scoring_rate"]
unclassed = rider_summary[rider_summary["rider_class"] == "Unclassed"]["scoring_rate"]

# Kruskal-Wallis test
h_stat, p_value = kruskal(all_rounder, climber, sprinter, unclassed)

# Calculate effect size (eta-squared)
n = len(rider_summary)
# k = number of groups(rider classes)
k = 4
eta_squared = (h_stat - k + 1) / (n - k)

print("\nKruskal-Wallis Test Results:")
print(f"H statistic: {h_stat:.4f}")
print(f"p-value: {p_value:.6f}")
print(f"Eta-squared: {eta_squared:.4f}")

alpha = 0.05
if p_value < alpha:
    print(f"Result: SIGNIFICANT (p < {alpha})")
    print("Conclusion: Reject Null Hypothesis, At least one rider class has different scoring rate")
else:
    print(f"Result: NOT SIGNIFICANT (p >= {alpha})")
    print("Conclusion: Accept Null Hypothesis, No evidence of scoring rate differences between classes")




Kruskal-Wallis Test Results:
H statistic: 28.1920
p-value: 0.000003
Epsilon-squared: 0.1400
Result: SIGNIFICANT (p < 0.05)
Conclusion: Reject Null Hypothesis, At least one rider class has different scoring rate


## Hypothesis B: Stage Class Specific Performance
- $H_0$: All rider classes have identical distributions of scoring rates on this stage class 
- $H_1$: At least one rider class has a different distribution of scoring rates on this stage
class

In [18]:
# Aggregate to rider-stage level
rider_stage = (
    df.groupby(["all_riders", "stage_class", "rider_class"])
    .agg(scoring_rate=("scored", "mean"))
    .reset_index()
)

# Bonferroni correction
n_tests = 3
alpha_bonf = 0.05 / n_tests
print(f"\nBonferroni-adjusted alpha: {alpha_bonf:.4f}")

hypothesis_results = []

stages = ["flat", "hills", "mount"]

for stage in stages:
    terrain_data = rider_stage[rider_stage["stage_class"] == stage]
    all_rounder_t = terrain_data[terrain_data["rider_class"] == "All Rounder"][
        "scoring_rate"
    ]
    climber_t = terrain_data[terrain_data["rider_class"] == "Climber"]["scoring_rate"]
    sprinter_t = terrain_data[terrain_data["rider_class"] == "Sprinter"]["scoring_rate"]
    unclassed_t = terrain_data[terrain_data["rider_class"] == "Unclassed"][
        "scoring_rate"
    ]

    h_stat_t, p_value_t = kruskal(all_rounder_t, climber_t, sprinter_t, unclassed_t)

    n_t = len(terrain_data)
    eta_squared_t = (h_stat_t - k + 1) / (n_t - k)

    print("\nKruskal-Wallis Test Results:")
    print(f"H statistic: {h_stat_t:.4f}")
    print(f"p-value: {p_value_t:.6f}")
    print(f"Eta-squared: {eta_squared_t:.4f}")

    if p_value_t < alpha_bonf:
        print(f"Result: SIGNIFICANT (p < {alpha_bonf:.4f})")
        conclusion = "At least one class differs"
    else:
        print(f"Result: NOT SIGNIFICANT (p >= {alpha_bonf:.4f})")
        conclusion = "No significant differences"

    print(f"Conclusion: {conclusion}")

    hypothesis_results.append(
        {
            "stage_class": stage,
            "H": h_stat_t,
            "p-value": p_value_t,
            "df": k - 1,
            "eta_squared": eta_squared_t,
            f"Decision at alpha_bonf({alpha_bonf:.4f})": "Reject null hypothesis"
            if p_value_t <= alpha_bonf
            else "Fail to reject null hypothesis",
        }
    )



Bonferroni-adjusted alpha: 0.0167

Kruskal-Wallis Test Results:
H statistic: 20.1749
p-value: 0.000156
Eta-squared: 0.0954
Result: SIGNIFICANT (p < 0.0167)
Conclusion: At least one class differs

Kruskal-Wallis Test Results:
H statistic: 27.5371
p-value: 0.000005
Eta-squared: 0.1363
Result: SIGNIFICANT (p < 0.0167)
Conclusion: At least one class differs

Kruskal-Wallis Test Results:
H statistic: 32.6824
p-value: 0.000000
Eta-squared: 0.1649
Result: SIGNIFICANT (p < 0.0167)
Conclusion: At least one class differs


In [19]:
# Summary table
stage_class_hypothesis = pd.DataFrame(hypothesis_results)
stage_class_hypothesis

,stage_class,H,p-value,df,eta_squared,Decision at alpha_bonf(0.0167)
0,flat,20.174887,1.561493e-04,3,0.095416,Reject null hypothesis
1,hills,27.537090,4.542406e-06,3,0.136317,Reject null hypothesis
2,mount,32.682420,3.757733e-07,3,0.164902,Reject null hypothesis
